In [ ]:
import numpy as np
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier


In [ ]:
from torch.nn.functional import pad
from underthesea import classify, word_tokenize, sent_tokenize
from pickle import load


max_len = 400

def extract_hidden_state(outputs):
    hiddenSize = outputs[0].size()[0]
    # flatten dimension 1 and 2, and hold batch dimension 0
    embeddings = outputs[0].view(hiddenSize, -1).numpy()
    return embeddings

def vectorize_mean(sentences):
    encodedText = tokenizer.batch_encode_plus(
        sentences, padding=True, truncation=True, max_length=300)['input_ids']
    embeddedTitle = phobert(torch.tensor(encodedText))
    with torch.no_grad():
        print('hiddenState', extract_hidden_state(embeddedTitle).shape)

    embedSqueeze = embeddedTitle['last_hidden_state'].squeeze(0)
    print('embedSqueeze', embedSqueeze.shape)
    embedMean = embedSqueeze.mean(axis=1)
    return embedMean.detach().numpy()

def trim_words(sentence, num_word=200):
    # Split the sentence into words
    words = sentence.split()
    # Trim the sentence to 5 words
    trimmed_sentence = " ".join(words[:num_word])
    return trimmed_sentence



X_train = vectorize_mean(train_df_check['text'])
y_train = train_df_check['labels'].values.tolist()
print('y_train', np.array(y_train).shape, X_train.shape)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

# Step 2: Create and fit the SVR model
model = SVR(kernel='linear')
model.fit(x_train, y_train.ravel())  # ravel y_train to convert to 1D array

# Step 5: Evaluate the model's performance (optional)
y_pred = model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")